# **ボストン住宅価格のデータから、価格を予測する機械学習モデルを作る**

データ解析練習用データセットの定番「ボストン住宅価格データ」をExcelにまとめました。

ボストン市内の490件の住宅に対し、
町ごとの犯罪率(CRIM)、雇用センターまでのの距離(DIS)、生徒教師比率(PTRATIO)・・・などなど、

住宅価格に影響しそうな条件と、実際の価格(Price)がまとめられています。

Excelファイルには、シートが２枚あり、

「For ML」シートには、490件の住宅の条件と価格

「For Prediction」シートには、予測したい住宅の条件（価格は未知）
が記載されています。


For MLのデータを元に、住宅価格を予測する機械学習モデルを作り、
For Predictionの条件に対する価格を予測しましょう。


Excelファイルはこちらから取得できます。
https://github.com/hatanaka-lab/Getting_started_with_MI/tree/main/data/Boston.xlsx


In [ ]:
#必要そうなライブラリたちをインポート
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#matplotlibで作成する図の中に日本語のフォントを利用する場合はこちらもインポート
!pip install japanize-matplotlib
import japanize_matplotlib

# **STEP1：データの読み込み**

In [ ]:
# Excelファイルの「For ML」Sheetからﾃﾞｰﾀを読み込んで、dfに代入
# dfは"data frame"の略。
# Pythonでは、"pandas"のdata frameと"numpy"のarrayの2種類をよく使うので、どちらか混乱しないように、data frameにはdfと名前を付ける癖をつけるとよい
df = pd.read_excel("Boston.xlsx", sheet_name='For ML')

In [ ]:
# dfの中身を確認①
# 各列の名前やﾃﾞｰﾀの個数（ぬけがないか等々）をcheck
df.info()

In [ ]:
# df の中身を確認②
# ﾃﾞｰﾀを上から３行だけ見てみる head(#) の#の数で表示する行数を指定
df.head(3)

# **STEP2：データを観察**

In [ ]:
# データの分布を観察する：その1
df_Target=df.drop(columns=['No'])
pd.plotting.scatter_matrix(df_Target, figsize=(12, 12), hist_kwds={'bins':10},
                           marker=('o'), s=8, alpha=.5)
plt.show()

In [ ]:
# データの分布を観察する：その2（相関行列）
correlation_coefficients = df_Target.corr()  # 相関行列の計算
# 相関行列のヒートマップ (相関係数の値あり)
plt.rcParams['font.size'] = 9
plt.figure(figsize=(12, 10))  # この段階で画像のサイズを指定する
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, annot=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.show()

# **STEP3：データを切り分け**

In [ ]:
# dfの1～14列の手前までをXに代入（説明変数に利用）
# 列数は、0からカウント。だから、Noが0列目、CRIMが1列目、LSTATが13行目
X = np.array(df.iloc[:,1:14])
df.iloc[:,1:14].head(2)  #中身の確認用

In [ ]:
# Priceとラベル付けされた列をYに代入（目的変数に利用）
Y = np.array(df["Price"])
df["Price"].head(2)  #中身の確認用

In [ ]:
#　説明変数を標準化する
#　標準化された値 = (元の値－平均値)/標準偏差
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# ホールドアウト検証用にﾃﾞｰﾀを　訓練ﾃﾞｰﾀ（train)と検証用ﾃﾞｰﾀ（test）に分ける。
# 分け方はランダム（今回は、train : test = 80% : 20%に分ける）
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2,random_state=999)

# **STEP4：ハイパーパラメタの決定**

In [ ]:
# RandomForest用にハイパーパラメタを決める
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
list_param = []
list_score = []
for ne in range(5,250,50):
  for nd in range(5,41,5):
    model = RandomForestRegressor(n_estimators=ne, max_depth=nd, random_state=555)
    cv5_score = cross_val_score(model, X_train, y_train, cv=5).mean()  #訓練用データ(train)を用いて5-fold CV
    print("num_trees=",ne,"max_depth=",nd,"R2_score=",cv5_score)
    list_param.append([ne,nd])
    list_score.append(cv5_score)
max_index = np.argmax(list_score)
print("")
print("-----Best parameters-----")
print("num_trees=",list_param[max_index][0], "max_depth=",list_param[max_index][1],"R2_score=",list_score[max_index])

# **STEP5：機械学習実行**

この直後に出てくる、RandomForestRegressorの部分を他の機械学習の方法名に置き換えると、色々な方法を試すことができる

In [ ]:
#　機械学習実行
#  Random Forest回帰を採用
#  これを実行すると、「model」の中身が 訓練ﾃﾞｰﾀから作られた機械学習モデルになる
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=155,max_depth=25,random_state=1)
model.fit(X_train, y_train)

In [ ]:
# 機械学習モデルの妥当性を検証
# 訓練ﾃﾞｰﾀとテストデータのスコアを見る　（scoreが1に近い程良いモデル）
print("Score for Training Data:", model.score(X_train,y_train))
print("Score for Test Data    :", model.score(X_test ,y_test ))

In [ ]:
# 散布図を描く
# 横軸が実際の値(y)：縦軸が機械学習モデルから予測した値
# データが対角線上にある程、予測精度が高いことを意味する
plt.figure(figsize=(4,4))
plt.scatter(y_train,model.predict(X_train),c='b',marker='o',alpha=0.7,label='Train')
plt.scatter(y_test, model.predict(X_test ),c='r',marker='^',alpha=0.7,label='Test')
# 対角線をひく
x1 = np.linspace(-1, 50, 100)
plt.plot(x1, x1, linestyle='-',c="silver")
#
plt.title("Random Forest 回帰", fontsize=14)
plt.ylabel("予測値", fontsize=12)
plt.xlabel("実際の値", fontsize=12)
plt.legend(fontsize=12)
plt.show()

横軸：実際の値、縦軸：機械学習による予測値。データが対角線上にあるほど、予測精度が良いことを示す。

# **STEP6：住宅価格が未知のデータに対する予測**

In [ ]:
# ここで、Priceの分からない未知のデータの説明変数（X)を読み込む
df_Pred = pd.read_excel("Boston.xlsx", sheet_name='For Prediction')

In [ ]:
# ﾃﾞｰﾀはX_Predに代入
X_Pred = np.array(df_Pred.iloc[:,1:14])
df_Pred.iloc[:,1:14].head(2)

In [ ]:
#　説明変数を標準化する
#  機械学習モデルを作った時に定義したscalerをそのまま適用
X_Pred_scaled = scaler.transform(X_Pred)

In [ ]:
#　機械学習モデルに入力して、Priceの値を算出
model.predict(X_Pred_scaled)

In [ ]:
# 上の表記だと、予測したいサンプルが多数ある場合に見にくいので、Excelに書き出す
df_result = pd.DataFrame(model.predict(X_Pred_scaled))
df_summary = pd.concat([df_Pred, df_result], axis=1)
df_summary.to_excel("Boston_result.xlsx")

In [ ]:
# ファイルダウンロード方法①
# Google Colab画面の左側の「ファイル」ボタンをクリック
# Boston_results.xlsxにカーソルを合わせ、右側の「縦に・が3つ並んだマーク」をクリック→ダウンロードをクリック

# ファイルダウンロード方法②
# 下記コードでもダウンロード可能
#from google.colab import files
#downloaded = files.download("Boston_result.xlsx")

ダウンロードファイルの一番右側の列に、予測価格が書き込まれています。

# **おまけ：他の機械学習モデルも色々検討**

例１：PLS回帰

In [ ]:
# RandomForest用にハイパーパラメタを決める
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
list_param = []
list_score = []
for k in range(2,13,1):
  model = PLSRegression(n_components=k)
  cv5_score = cross_val_score(model, X_train, y_train, cv=5).mean()  #訓練用データ(train)を用いて5-fold CV
  print("n_components=", k, "R2_score=", cv5_score)
  list_param.append(k)
  list_score.append(cv5_score)
max_index = np.argmax(list_score)
print("")
print("-----Best parameters-----")
print("n_components=",list_param[max_index],"R2_score=",list_score[max_index])

In [ ]:
# 機械学習の方法を選択
# 今回はPLS回帰（個人的にはお気に入り）
from sklearn.cross_decomposition import PLSRegression
model = PLSRegression(n_components=8)

# 機械学習実行：ここから後ろは、RandomForestの時と変更なし
model.fit(X_train, y_train)
# 検証
print("Score for Training Data:", model.score(X_train,y_train))
print("Score for Test Data    :", model.score(X_test ,y_test ))
# 散布図描画
plt.figure(figsize=(4,4))
plt.scatter(y_train,model.predict(X_train),c='b',marker='o',alpha=0.7,label='Train')
plt.scatter(y_test, model.predict(X_test ),c='r',marker='^',alpha=0.7,label='Test')
x1 = np.linspace(-1, 50, 100)
plt.plot(x1, x1, linestyle='-',c="silver")
plt.title("PLS回帰", fontsize=14)
plt.ylabel("予測値", fontsize=12)
plt.xlabel("実際の値", fontsize=12)
plt.legend(fontsize=12)
plt.show()

例２：LASSO回帰

In [ ]:
# 機械学習の方法を選択
# LASSO回帰のハイパーパラメタチューニングと回帰を両方実施（親切♪）
from sklearn.linear_model import LassoCV
model = LassoCV(n_alphas=50, cv=5, max_iter=100000)

# 機械学習実行：ここから後ろは、RandomForestの時と変更なし
model.fit(X_train, y_train)
# 5-fold CVで決めたハイパーパラメタ(α)を表示
print('alpha    = ', model.alpha_)
# 検証
print("Score for Training Data:", model.score(X_train,y_train))
print("Score for Test Data    :", model.score(X_test ,y_test ))
# 散布図描画
plt.figure(figsize=(4,4))
plt.scatter(y_train,model.predict(X_train),c='b',marker='o',alpha=0.7,label='Train')
plt.scatter(y_test, model.predict(X_test ),c='r',marker='^',alpha=0.7,label='Test')
x1 = np.linspace(-1, 50, 100)
plt.plot(x1, x1, linestyle='-',c="silver")
plt.title("LASSO回帰", fontsize=14)
plt.ylabel("予測値", fontsize=12)
plt.xlabel("実際の値", fontsize=12)
plt.legend(fontsize=12)
plt.show()